In [ ]:
!pip install -q google-genai pyarrow

from google.colab import auth
auth.authenticate_user()  # OAuth in Colab

PROJECT_ID = "620323772193"   # <- set this
LOCATION  = "us-central1"            # recommended

import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

from vertexai.language_models import TextEmbeddingModel
model = TextEmbeddingModel.from_pretrained("text-embedding-004")  # Vertex text embeddings
print("Vertex ready:", PROJECT_ID, LOCATION)

/usr/local/lib/python3.12/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Vertex ready: 620323772193 us-central1


In [ ]:
import pandas as pd
import numpy as np
import math, time, random
from typing import List
from tqdm import tqdm

# IO paths
DATA_IN  = "/content/cleaned_data.parquet"   # change if CSV
EMB_OUT  = "embeddings.parquet"

# Embedding config
BATCH_SIZE   = 100       # Vertex handles ~100 nicely
TASK_TYPE    = "RETRIEVAL_DOCUMENT"  # good default for recsys search
MAX_RETRIES  = 6
BASE_SLEEP_S = 1.2

print("Config OK:", {"BATCH_SIZE": BATCH_SIZE, "TASK_TYPE": TASK_TYPE})

Config OK: {'BATCH_SIZE': 100, 'TASK_TYPE': 'RETRIEVAL_DOCUMENT'}


In [ ]:
# If your file is CSV, use: pd.read_csv("clean_anime_dataset.csv")
df = pd.read_parquet(DATA_IN)

print("Rows:", len(df))
print("Columns:", list(df.columns))
display(df.head(3))

Rows: 8857
Columns: ['anime_id', 'score', 'genres', 'themes', 'demographics', 'synopsis', 'type', 'episodes', 'premiered', 'producers', 'studios', 'source', 'duration', 'rating', 'popularity', 'members']


,anime_id,score,genres,themes,demographics,synopsis,type,episodes,premiered,producers,studios,source,duration,rating,popularity,members
0,52991,9.29,adventure drama fantasy,nan,shounen,During their decade-long quest to defeat the D...,tv,28.0,2023.0,aniplex dentsu shogakukan shueisha productions...,madhouse,manga,24.0,pg 13 teens 13 or older,150.99,1112018.75
1,5114,9.10,action adventure drama fantasy,military,shounen,After a horrific alchemy experiment goes wrong...,tv,64.0,2009.0,aniplex square enix mainichi broadcasting syst...,bones,manga,24.0,r 17 violence profanity,150.99,1112018.75
2,9253,9.07,drama sci fi suspense,psychological time travel,nan,Eccentric scientist Rintarou Okabe has a never...,tv,24.0,2011.0,frontier works media factory kadokawa shoten m...,white fox,visual novel,24.0,pg 13 teens 13 or older,150.99,1112018.75


In [ ]:
MAX_SYNOPSIS_CHARS = 1200  # keep it focused; adjust if needed

def _to_list(x):
    if x is None:
        return []
    if isinstance(x, list):
        return [str(v).strip() for v in x if str(v).strip()]
    if isinstance(x, str):
        return [s.strip() for s in x.split(",") if s.strip()]
    s = str(x).strip()
    return [s] if s else []

def make_genre_theme_set(row):
    bag = _to_list(row.get("genres")) + _to_list(row.get("themes")) + _to_list(row.get("demographics"))
    return sorted(set(bag))

df["genre_theme_set"] = df.apply(make_genre_theme_set, axis=1)

def make_embed_text(row):
    synopsis = (row.get("synopsis") or "").replace("\n", " ").strip()
    if len(synopsis) > MAX_SYNOPSIS_CHARS:
        synopsis = synopsis[:MAX_SYNOPSIS_CHARS].rsplit(" ", 1)[0] + "..."
    gts = " ".join(row.get("genre_theme_set", []))
    return f"Genres: {gts}. {synopsis}"

df["embed_text"] = df.apply(make_embed_text, axis=1)

print("Sample compact embed_text rows:")
display(df[["anime_id", "genre_theme_set", "embed_text"]].head(3))

Sample compact embed_text rows:


,anime_id,genre_theme_set,embed_text
0,52991,"[adventure drama fantasy, nan, shounen]",Genres: adventure drama fantasy nan shounen. D...
1,5114,"[action adventure drama fantasy, military, sho...",Genres: action adventure drama fantasy militar...
2,9253,"[drama sci fi suspense, nan, psychological tim...",Genres: drama sci fi suspense nan psychologica...


In [ ]:
df.to_parquet("cleaned_data2.parquet", index=False)
print("Saved the current DataFrame to cleaned_data2.parquet")

Saved the current DataFrame to cleaned_data2.parquet


In [ ]:
import math, time
from tqdm import tqdm
import numpy as np

# Heuristic: ~4 characters ≈ 1 token (safe-ish). Use 18k to leave headroom under the 20k hard cap.
CHARS_PER_TOKEN = 4
MAX_TOKENS_PER_REQUEST = 18_000
HARD_CAP_BATCH = 100  # never exceed this many rows per request even if token budget remains

def _estimate_tokens(text: str) -> int:
    # Fast heuristic; Vertex CountTokens API exists, but we avoid extra round trips.
    return max(1, len(text) // CHARS_PER_TOKEN)

def _pack_batches_by_budget(texts):
    batches, cur, cur_tokens = [], [], 0
    for t in texts:
        est = _estimate_tokens(t)
        # if adding this would exceed budget or hard cap, flush the batch
        if cur and (cur_tokens + est > MAX_TOKENS_PER_REQUEST or len(cur) >= HARD_CAP_BATCH):
            batches.append(cur)
            cur, cur_tokens = [], 0
        # if a single text is huge, still send it alone (SDK can auto-truncate per item)
        cur.append(t)
        cur_tokens += est
    if cur:
        batches.append(cur)
    return batches

def _call_get_embeddings(batch):
    # Prefer auto_truncate=True if this SDK supports it
    try:
        return model.get_embeddings(batch, auto_truncate=True)
    except TypeError:
        return model.get_embeddings(batch)

def get_embeddings_batched(texts, max_retries=6, base_sleep=1.2):
    batches = _pack_batches_by_budget(texts)
    print(f"Prepared {len(batches)} batches (token-budgeted).")
    out = []
    for i, batch in enumerate(tqdm(batches, total=len(batches))):
        attempt = 0
        while True:
            try:
                resp = _call_get_embeddings(batch)
                out.extend([np.asarray(e.values, dtype=np.float32) for e in resp])
                break
            except Exception as e:
                if attempt >= max_retries:
                    raise RuntimeError(f"Failed after {max_retries} retries on batch {i} (size={len(batch)}): {e}")
                wait = base_sleep * (2 ** attempt)
                print(f"Retrying batch {i} (size={len(batch)}) in {wait:.1f}s due to: {e}")
                time.sleep(wait)
                attempt += 1
    return np.vstack(out, dtype=np.float32)

texts = df["embed_text"].fillna("").astype(str).tolist()
emb = get_embeddings_batched(texts)

print("Embedding shape:", emb.shape, "| dtype:", emb.dtype)
print("First row norm (pre-norm):", float(np.linalg.norm(emb[0])))

Prepared 93 batches (token-budgeted).


100%|██████████| 93/93 [02:10<00:00,  1.40s/it]

Embedding shape: (8857, 768) | dtype: float32
First row norm (pre-norm): 0.9999672770500183


In [ ]:
def l2_normalize(arr: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(arr, axis=1, keepdims=True)
    n[n == 0.0] = 1.0
    return arr / n

emb = l2_normalize(emb)
assert emb.shape[0] == len(df), "Row count mismatch"
assert emb.shape[1] in (512, 768, 1024), "Unexpected embedding dimension"
assert np.isfinite(emb).all(), "Found NaN/Inf in embeddings"

out = pd.DataFrame({
    "anime_id": df["anime_id"].values,
    "embedding": list(emb),                 # parquet-friendly (list per row)
    "genre_theme_set": df["genre_theme_set"].values,
})

out.to_parquet(EMB_OUT, index=False)
print(f"Saved {len(out)} vectors → {EMB_OUT} (dim={emb.shape[1]})")
print("Example row:")
display(out.head(1))

✅ Saved 8857 vectors → embeddings.parquet (dim=768)
Example row:


,anime_id,embedding,genre_theme_set
0,52991,"[-0.018555958, 0.029415023, -0.06058112, 0.016...","[adventure drama fantasy, nan, shounen]"


In [ ]:
testdf = pd.read_parquet(EMB_OUT)
print(f"Loaded {len(testdf)} vectors from {EMB_OUT}")
display(testdf.head(3))

Loaded 8857 vectors from embeddings.parquet


,anime_id,embedding,genre_theme_set
0,52991,"[-0.018555958, 0.029415023, -0.06058112, 0.016...","[adventure drama fantasy, nan, shounen]"
1,5114,"[-0.0149866035, 0.024781926, -0.0836438, 0.018...","[action adventure drama fantasy, military, sho..."
2,9253,"[-0.023494612, 0.018279973, -0.03611847, 0.046...","[drama sci fi suspense, nan, psychological tim..."


In [ ]:
import re
import pandas as pd
import numpy as np
from pathlib import Path

SEQUEL_WORDS = {
    'season','seasons','part','parts','cour','cours','ova','ovas','special',
    'movie','movies','final','chapter','chapters','arc','arcs','remake','reboot',
    'specials','episode','episodes','tv','the','animation','edition','editions'
}
ROMAN = r'(?:[ivxlcdm]+)'
SEP = r'[:\-–—]|(?<=\s)\((?:[^)]*)\)|\[(?:[^\]]*)\]'
NUM_OR_ROMAN_TAIL = r'\s+(?:' + ROMAN + r'|\d+)$'

def _basic_norm(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(SEP, ' ', s)               # cut subtitles and bracketed notes
    s = re.sub(r'[^\w\s/]+', ' ', s)      # strip punctuation except '/'
    s = re.sub(NUM_OR_ROMAN_TAIL, '', s)  # drop trailing numerals/roman
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def _drop_sequel_words(s: str) -> str:
    toks = [t for t in s.split() if t not in SEQUEL_WORDS]
    # drop trailing single digits/roman again if left
    while toks and (re.fullmatch(r'\d+|' + ROMAN, toks[-1]) is not None):
        toks.pop()
    return ' '.join(toks)

def _series_key_from_one(name: str) -> str:
    if not isinstance(name, str) or not name.strip():
        return ''
    s = _basic_norm(name)
    s = _drop_sequel_words(s)
    s = s.split('/')[0].strip()           # keep left side of things like "fate/stay night"
    head = ' '.join(s.split()[:4])        # 3–5 tokens is a good head; start with 4
    return head

def compute_series_key_row(row) -> str:
    candidates = []
    for col in ('name', 'english_name', 'japanese_names'):
        val = row.get(col, None)
        if isinstance(val, str) and val.strip():
            parts = [p.strip() for p in re.split(r'[;|,/]', val) if p.strip()]
            for p in parts or [val]:
                key = _series_key_from_one(p)
                if key:
                    candidates.append(key)
    if not candidates:
        return ''
    counts = pd.Series(candidates).value_counts()
    top_count = counts.iloc[0]
    ties = [k for k, v in counts.items() if v == top_count]
    return min(ties, key=len)  # prefer shorter among the most frequent

# We still have `df` (full metadata) and `out` (embeddings table) in memory from previous cells.
# Use `new_df` which has the original columns
# series_df = new_df[['anime_id','name','english_name','japanese_names']].copy()
# series_df['series_key'] = series_df.apply(compute_series_key_row, axis=1)

# Merge onto the embeddings table we just wrote
# out_with_series = out.merge(series_df[['anime_id','series_key']], on='anime_id', how='left')
# Assuming 'out' already has 'series_key' from previous runs or is not needed for now
out_with_series = out # Use 'out' directly if series_key is not being regenerated

# Save a new file alongside EMB_OUT
EMB_OUT_SER = str(Path(EMB_OUT).with_name(Path(EMB_OUT).stem + "_series.parquet"))
out_with_series.to_parquet(EMB_OUT_SER, index=False)

print(f"🔑 Added series_key → {EMB_OUT_SER}")
display(out_with_series.head(3))

🔑 Added series_key → embeddings_series.parquet


,anime_id,embedding,genre_theme_set
0,52991,"[-0.018555958, 0.029415023, -0.06058112, 0.016...","[adventure drama fantasy, nan, shounen]"
1,5114,"[-0.0149866035, 0.024781926, -0.0836438, 0.018...","[action adventure drama fantasy, military, sho..."
2,9253,"[-0.023494612, 0.018279973, -0.03611847, 0.046...","[drama sci fi suspense, nan, psychological tim..."


In [ ]:
import pandas as pd
import numpy as np

# --- Load the CSV lookup file once ---
raw_data = pd.read_csv("raw_data.csv")  # must contain anime_id, name, score, episodes columns
raw_data["anime_id"] = pd.to_numeric(raw_data["anime_id"], errors="coerce")

import pandas as pd
import numpy as np

# --- Load the CSV lookup file once ---
raw_data = pd.read_csv("raw_data.csv")  # must contain anime_id, name, score, episodes columns
raw_data["anime_id"] = pd.to_numeric(raw_data["anime_id"], errors="coerce")

def recommend_with_diversity(anime_ids: list[int],
                             top_k_candidates: int = 100,
                             final_k: int = 10):
    """
    Recommend similar anime using the *average* embedding of up to 3 anime IDs.
    Uses dot-product similarity and genre diversity (no MMR or title-based filtering).

    Parameters
    ----------
    anime_ids : list[int]
        A list of 1–3 anime IDs to use as the query.
    top_k_candidates : int
        Number of top candidates to consider before re-ranking.
    final_k : int
        Number of final recommendations to display.
    """

    # --- 1) Validate input ---
    if not isinstance(anime_ids, (list, tuple)):
        raise ValueError("anime_ids must be a list or tuple of integers.")
    if len(anime_ids) == 0 or len(anime_ids) > 3:
        raise ValueError("You must provide between 1 and 3 anime IDs.")

    if "anime_id" not in out.columns:
        raise ValueError("The 'out' DataFrame must contain an 'anime_id' column.")

    # --- 2) Locate the matching rows ---
    indices = []
    for aid in anime_ids:
        hits = out.index[out["anime_id"] == aid].tolist()
        if not hits:
            print(f"⚠️ Anime ID {aid} not found — skipping.")
            continue
        indices.append(hits[0])

    if not indices:
        raise ValueError("None of the provided anime IDs were found in the dataset.")

    # --- 3) Build query vector (average of up to 3 embeddings) ---
    qv = np.mean(emb[indices], axis=0)

    # --- 4) Compute dot product similarity ---
    sims = np.dot(emb, qv)
    K = min(top_k_candidates, len(out))
    cand = np.argpartition(-sims, K)[:K]
    cand = cand[np.argsort(-sims[cand])]

    # Remove all query anime themselves
    cand = [j for j in cand if j not in indices]

    # --- 5) Genre diversity filtering ---
    chosen = cand[:final_k * 3]  # take slightly more for variety
    used_genres, diverse = set(), []

    for j in chosen:
        genres_field = out.iloc[j].get("genre_theme_set", [])
        if isinstance(genres_field, str):
            genres = [genres_field]
        elif isinstance(genres_field, (list, np.ndarray)):
            genres = [str(g) for g in genres_field if g]
        else:
            genres = []
        main_genre = genres[0] if genres else "Unknown"
        if main_genre not in used_genres:
            diverse.append(j)
            used_genres.add(main_genre)
        if len(diverse) >= final_k:
            break

    # Fill remaining spots if needed
    if len(diverse) < final_k:
        remaining = [x for x in chosen if x not in diverse]
        diverse.extend(remaining[:final_k - len(diverse)])
        diverse = diverse[:final_k]

    # --- 6) Print recommendations ---
    print(f"Query anime_ids: {anime_ids}")
    for j in diverse:
        target_id = out.iloc[j].get("anime_id", None)
        if target_id is None:
            continue

        info = raw_data.loc[raw_data["anime_id"] == target_id]
        if not info.empty:
            name = info.iloc[0].get("name", "Unknown")
            score = info.iloc[0].get("score", "N/A")
            episodes = info.iloc[0].get("episodes", "N/A")
        else:
            name, score, episodes = "Unknown", "N/A", "N/A"

        print(f"{target_id:<8}  {name:<45} | sim={sims[j]:.4f} | score={score} | episodes={episodes}")


recommend_with_diversity(anime_ids=[269, 136, 20])


Query anime_ids: [269, 136, 20]
11061     Hunter x Hunter (2011)                        | sim=0.7747 | score=9.03 | episodes=148.0
53998     Bleach: Sennen Kessen-hen - Ketsubetsu-tan    | sim=0.7442 | score=8.7 | episodes=13.0
16870     The Last: Naruto the Movie                    | sim=0.7064 | score=7.79 | episodes=1.0
7592      Nurarihyon no Mago                            | sim=0.6947 | score=7.6 | episodes=24.0
10686     Naruto: Honoo no Chuunin Shiken! Naruto vs. Konohamaru!! | sim=0.6918 | score=7.17 | episodes=1.0
43760     Hikari no Ou                                  | sim=0.6827 | score=6.32 | episodes=10.0
356       Fate/stay night                               | sim=0.6822 | score=7.27 | episodes=24.0
1735      Naruto: Shippuuden                            | sim=0.7491 | score=8.28 | episodes=500.0
28755     Boruto: Naruto the Movie                      | sim=0.7427 | score=7.37 | episodes=1.0
41467     Bleach: Sennen Kessen-hen                     | sim=0.7414 | score=8